## Set up

In [2]:
# install
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
%sql CREATE SCHEMA IF NOT EXISTS kindle

 * Starting PostgreSQL 14 database server
   ...done.
CREATE ROLE
 * postgresql+psycopg2://@/postgres


""


In [3]:
import pandas as pd
import sqlalchemy

engine = sqlalchemy.create_engine("postgresql+psycopg2://@/postgres")
conn = engine.connect()


In [4]:
from google.colab import drive
drive.mount('/content/drive')
ruta = '/content/drive/MyDrive/Datasets'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data = pd.read_csv(ruta + '/kindle_data-v2.csv')

In [6]:
data.to_sql("data",engine,if_exists="append",method="multi",schema="kindle",index=False)

133102

In [7]:
data.columns

Index(['asin', 'title', 'author', 'soldBy', 'imgUrl', 'productURL', 'stars',
       'reviews', 'price', 'isKindleUnlimited', 'category_id', 'isBestSeller',
       'isEditorsPick', 'isGoodReadsChoice', 'publishedDate', 'category_name'],
      dtype='object')

In [8]:
%%sql

SELECT * FROM kindle.data


 * postgresql+psycopg2://@/postgres


,asin,title,author,soldBy,imgUrl,productURL,stars,reviews,price,isKindleUnlimited,category_id,isBestSeller,isEditorsPick,isGoodReadsChoice,publishedDate,category_name
0,B00TZE87S4,Adult Children of Emotionally Immature Parents...,Lindsay C. Gibson,Amazon.com Services LLC,https://m.media-amazon.com/images/I/713KZTsaYp...,https://www.amazon.com/dp/B00TZE87S4,4.8,0,9.99,False,6,True,False,False,2015-06-01,Parenting & Relationships
1,B08WCKY8MB,"From Strength to Strength: Finding Success, Ha...",Arthur C. Brooks,Penguin Group (USA) LLC,https://m.media-amazon.com/images/I/A1LZcJFs9E...,https://www.amazon.com/dp/B08WCKY8MB,4.4,0,16.99,False,6,False,False,False,2022-02-15,Parenting & Relationships
2,B09KPS84CJ,Good Inside: A Guide to Becoming the Parent Yo...,Becky Kennedy,HarperCollins Publishers,https://m.media-amazon.com/images/I/71RIWM0sv6...,https://www.amazon.com/dp/B09KPS84CJ,4.8,0,16.99,False,6,False,True,False,2022-09-13,Parenting & Relationships
3,B07S7QPG6J,Everything I Know About Love: A Memoir,Dolly Alderton,HarperCollins Publishers,https://m.media-amazon.com/images/I/71QdQpTiKZ...,https://www.amazon.com/dp/B07S7QPG6J,4.2,0,9.95,True,6,False,True,False,2020-02-25,Parenting & Relationships
4,B00N6PEQV0,The Seven Principles for Making Marriage Work:...,John Gottman,Random House LLC,https://m.media-amazon.com/images/I/813o4WOs+w...,https://www.amazon.com/dp/B00N6PEQV0,4.7,0,13.99,False,6,False,False,False,2015-05-05,Parenting & Relationships
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133097,B0C3SJDLK8,Anna Karenina,Leo Tolstoy,De Marque,https://m.media-amazon.com/images/I/61RXEyMctk...,https://www.amazon.com/dp/B0C3SJDLK8,4.7,0,0.00,False,29,True,False,False,2023-09-15,Nonfiction
133098,B09DW752Y1,Scary Smart: Scary Smart: The Future of Artifi...,Mo Gawdat,Amazon.com Services LLC,https://m.media-amazon.com/images/I/71tjf6R-Wa...,https://www.amazon.com/dp/B09DW752Y1,4.4,0,9.99,False,29,False,False,False,2021-09-30,Nonfiction
133099,B0CFWHPJTN,The Count of Monte Cristo: The Original Unabri...,Alexandre Dumas,De Marque,https://m.media-amazon.com/images/I/71zcCb5Pvu...,https://www.amazon.com/dp/B0CFWHPJTN,4.8,0,1.99,False,29,True,False,False,2023-09-15,Nonfiction
133100,B0C1KMFXJC,War And Peace,Leo Tolstoy,De Marque,https://m.media-amazon.com/images/I/715QpZlzmi...,https://www.amazon.com/dp/B0C1KMFXJC,4.7,0,0.00,False,29,True,False,False,2023-08-25,Nonfiction


In [9]:
%%sql

SELECT count(title) "cantidad total de titulos" , count(distinct title) "cantidad de titulos unicos"
FROM kindle.data;

 * postgresql+psycopg2://@/postgres


,cantidad total de titulos,cantidad de titulos unicos
0,133102,131913


The 10 books with more stars


In [10]:
%%sql

SELECT DISTINCT  title, author, stars
FROM kindle.data
WHERE stars > (SELECT AVG(stars) FROM kindle.data)
ORDER BY stars DESC LIMIT 10

 * postgresql+psycopg2://@/postgres


,title,author,stars
0,$100M Leads: How to Get Strangers To Want To B...,Alex Hormozi,5.0
1,"10,000 Startups: Legal Strategies For Startup ...",Roger Royse,5.0
2,100 Exercises for Practicing Python: A set of ...,Laurentine K.Masson,5.0
3,100 Hikes: Eastern Oregon (Oregon Guidebooks),William Sullivan,5.0
4,100 Hikes: Northwest Oregon (Oregon Guidebooks),William L. Sullivan,5.0
5,100 Interview Questions: C++ (Advanced Topics ...,X.Y. Wang,5.0
6,100 IS THE NEW 30: HOW PLAYING THE SYMPHONY OF...,JEFFREY GLADDEN MD FACC,5.0
7,+100 Self Love Book for Women. Self-Help with ...,Ivi Green,5.0
8,102 Most Useful Excel Functions with Examples:...,Steve B.McAlister,5.0
9,10 Artist Commandments: 10 Principles to Creat...,J.R. Mckee,5.0


The 10 books with most reviews in kindle

In [11]:
%%sql
SELECT DISTINCT  title, author, reviews
FROM kindle.data
ORDER BY reviews DESC LIMIT 10

 * postgresql+psycopg2://@/postgres


,title,author,reviews
0,Where the Crawdads Sing,Delia Owens,618227
1,It Ends with Us: A Novel,Colleen Hoover,296710
2,The Nightingale: A Novel,Kristin Hannah,289251
3,"The Silent Patient: The record-breaking, multi...",Alex Michaelides,272608
4,Reminders of Him: A Novel,Colleen Hoover,242575
5,The Midnight Library: A Novel,Matt Haig,234933
6,Eleanor Oliphant Is Completely Fine: A Novel,Gail Honeyman,227722
7,Beneath a Scarlet Sky: A Novel,Mark Sullivan,223114
8,Lessons in Chemistry: A Novel,Bonnie Garmus,219990
9,All the Light We Cannot See: A Novel,Anthony Doerr,216532


The 15 books with the highest prices in kindle

In [12]:
%%sql
SELECT DISTINCT  title, author, price
FROM kindle.data
ORDER BY price DESC LIMIT 15

 * postgresql+psycopg2://@/postgres


,title,author,price
0,Drugs in Litigation: Damage Awards Involving P...,LexisNexis Editorial Staff,682.00
1,Broker-Dealer Regulation,Clifford E. Kirsch,662.00
2,Youmans and Winn Neurological Surgery E-Book: ...,H. Richard Winn,636.99
3,How to Write a Patent Application,Jeffrey G. Sheldon,629.00
4,The Collected Works of C. G. Jung: Revised and...,C. G. Jung,549.99
5,"The Art of Aesthetic Surgery, Three Volume Set...",Foad Nahai,543.99
6,"Perforator Flaps: Anatomy, Technique, & Clinic...",Phillip N. Blondeel,481.49
7,International Commercial Arbitration: Three Vo...,Gary B. Born,480.00
8,LexisNexis Practice Guide: Florida Personal In...,Ervin A. Gonzalez,465.99
9,Private Equity Funds: Formation and Operations,Stephanie R. Breslow,462.00


In [13]:
%%sql
SELECT DISTINCT  title, author, price FROM kindle.data
WHERE price > 0
ORDER BY price ASC LIMIT 10

 * postgresql+psycopg2://@/postgres


,title,author,price
0,Discourses on Livy,Niccolò Machiavelli,0.50
1,The Code of Hammurabi,Leonard William King,0.50
2,Dónde descargar todos los libros gratis para K...,Carlota Garrido,0.69
3,How to Own Your Own Mind: Unlock Your Inner Po...,Napoleon Hill,0.75
4,1000 Facts about Historic Figures Vol. 1,James Egan,0.99
5,1000 Facts about Historic Figures Vol. 3,James Egan,0.99
6,1001 Best Baking Recipes of All Time: A Baking...,Emma Katie,0.99
7,1001 Best Crock Pot Recipes of All Time: A Cro...,Emma Katie,0.99
8,100 Puzzle Quizzes (Interactive Puzzlebook for...,The Grabarchuk Family,0.99
9,101 Essential Life Skills For Teens: How to Be...,J.R. Beel,0.99


Total titles by category

In [12]:
%%sql

SELECT category_name, COUNT(title) AS "Total titles"
FROM kindle.data
GROUP BY category_name
ORDER BY COUNT(title) DESC


 * postgresql+psycopg2://@/postgres


,category_name,Total titles
0,"Mystery, Thriller & Suspense",18816
1,Engineering & Transportation,18486
2,Science & Math,17865
3,Parenting & Relationships,17703
4,Children's eBooks,17241
5,Teen & Young Adult,17214
6,Foreign Language,17172
7,Travel,16578
8,LGBTQ+ eBooks,16551
9,Computers & Technology,16512


The top 10 Authors with the Most Published Titles

In [37]:
%%sql

SELECT DISTINCT author, COUNT(title) AS "Total titles"
FROM kindle.data
WHERE author != 'None'
GROUP BY author
ORDER BY COUNT(title) DESC
LIMIT 10

 * postgresql+psycopg2://@/postgres


,author,Total titles
0,James Patterson,212
1,DK Eyewitness,163
2,DK,155
3,Captivating History,120
4,Fodor's Travel Guides,115
5,Erin Hunter,114
6,J.K. Rowling,113
7,Hourly History,103
8,America's Test Kitchen,99
9,Nora Roberts,99


The top 10 of free books with more stars and reviews

In [38]:
%%sql

SELECT DISTINCT  title, author, stars, reviews
FROM kindle.data
WHERE price = 0
ORDER BY stars DESC, reviews DESC
LIMIT 10


 * postgresql+psycopg2://@/postgres


,title,author,stars,reviews
0,Ignite Your Courage: Inspiring Moments of Unto...,JB Owen,5.0,68
1,The White Tablets of Melchizedek: The Codes of...,Christina Rice,5.0,43
2,Legacy By Design Rewiring Your Mind For Success,LaTonya Renee,5.0,37
3,The Underdog Founder: How to Go From Unseen to...,Edrizio De La Cruz,5.0,35
4,Chicken Coop Pinky Swear: The Impossible Promi...,Peter Andrew Meerwarth,5.0,33
5,You Are Not Here: Travels Through Countries th...,Eric Czuleger,5.0,33
6,Winning Tools: 3 Leadership Principles That Bu...,Matthew Mitchell,5.0,31
7,¿Ahora Que? : Los errores no son el final (Spa...,Reynaldo Rivera Cancel,5.0,25
8,Marketing Magic: How I produced 7 billion view...,Manuel Suarez,5.0,25
9,Expert Resilience: How Entrepreneurs Are Leadi...,Rob Kosberg,5.0,22


The 10 top-rated books of 2023.

In [36]:
%%sql

SELECT title, author, stars, reviews, "publishedDate"
FROM kindle.data
WHERE "publishedDate" >'01/01/2023'
ORDER BY stars DESC, reviews DESC
LIMIT 10

 * postgresql+psycopg2://@/postgres


,title,author,stars,reviews,publishedDate
0,Cold-Case Christianity (Updated & Expanded Edi...,J. Warner Wallace,5.0,527,2023-09-05
1,Super Easy Crock Pot Cookbook for Beginners: 1...,Deborah P. Gunter,5.0,433,2023-08-30
2,The First Happiness System: A Self-Help Book t...,Oleksandr Suvorov,5.0,424,2023-08-08
3,Greece: Athens & The Peloponnese Travel Guide ...,Dreamscape Adventures,5.0,318,2023-08-05
4,A Beginner's Guide to ChatGPT: Leveraging Arti...,Robert Scales,5.0,266,2023-04-18
5,The ChatGPT Advantage: Transform Your Business...,Diana Sterling,5.0,247,2023-07-26
6,Python Programming and SQL: [7 in 1] The Most ...,Pharell Hearst,5.0,230,2023-06-05
7,System Design Interview for Beginners: Master ...,Paul Lin,5.0,230,2023-09-02
8,LOW FODMAP DIET GUIDE AND COOKBOOK: Alleviate ...,Emma Greenfield,5.0,197,2023-08-29
9,Juicing For Two: Embrace the Art of Juicing To...,Keri Fox,5.0,181,2023-07-11


The 10 top-rated Best Sellers books

In [45]:
%%sql

SELECT title, author, stars, reviews
FROM kindle.data
WHERE "isBestSeller" = 'True'
ORDER BY stars DESC, reviews DESC
LIMIT 20

 * postgresql+psycopg2://@/postgres


,title,author,stars,reviews
0,Cold-Case Christianity (Updated & Expanded Edi...,J. Warner Wallace,5.0,527
1,Selective Breeding and the Birth of Philosophy,Costin Alamariu,5.0,67
2,If Only I'd Known: How to Outsmart Narcissists...,Chelsey Brooke Cole,5.0,47
3,Shadow Work Journal: Break Free from Self-Sabo...,Bob Barret,5.0,43
4,Course Creation Simplified: The 6-Phase System...,Jimmy Naraine,5.0,35
5,Winning Tools: 3 Leadership Principles That Bu...,Matthew Mitchell,5.0,31
6,"After the Storm: A True Story of Tragedy, Surv...",Dr. Lauren Farmer,5.0,22
7,Beyond the Border: A Korean's Journey Between ...,Tae-hyok Kim,5.0,16
8,A Masterpiece in Disarray: David Lynch’s Dune....,Max Evry,5.0,15
9,The Tenderness of Jesus: An Invitation to Expe...,Rob Reimer,5.0,13
